In [1]:
import random
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

NameError: name 'sc' is not defined

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [13]:
from typing import List
a: List[int] = ['1']

In [32]:
class A(object):
    def help(self):
        self: A = self
    
print(dir(A))
a = A()
type(a).__name__

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'help']


'A'

## 读取本地文件，创建临时表

使用 sql 查询，统计，可视化

In [2]:

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Test") \
    .config('spark.default.parallelism','2') \
    .config('spark.executor.cores','2') \
    .getOrCreate()

df = spark.read.format("com.databricks.spark.csv") \
.option("delimiter", "\t") \
.load("/Users/andyzhuang/git/maps_arch/road_sampling/data/grid_length.csv")

df.show(5)
df.createTempView("tempViewName") 

+-------------+-----+------+------+----+
|          _c0|  _c1|   _c2|   _c3| _c4|
+-------------+-----+------+------+----+
| 310231212030|35874|610429|陕西省|西北|
|3102312112312| 9767|610528|陕西省|西北|
|3102312112310|19650|610528|陕西省|西北|
|3102312112313|25445|610528|陕西省|西北|
|3102312112311| 6825|610528|陕西省|西北|
+-------------+-----+------+------+----+
only showing top 5 rows



In [16]:

groupby_df1 = spark.sql("select count(1) c,_c3 region,sum(_c1) dist from tempViewName group by _c3 order by dist")
groupby_df1.show()
rows = groupby_df1.collect()
region_list = []
dist_list = []
for r in rows:
    region_list.append(r.region)
    dist_list.append(r.dist)

# import matplotlib.pyplot as plt 
# plt.rcParams['font.family'] = ['sans-serif']
# plt.rcParams['font.sans-serif'] = ['Helvetica']
# plt.bar(region_list, dist_list)
# plt.show()

+----+--------------+-----------+
|   c|        region|       dist|
+----+--------------+-----------+
|   1|澳门特别行政区|    16778.0|
| 924|        海南省| 1.450653E7|
|1487|    西藏自治区|1.4597324E7|
|1266|宁夏回族自治区|2.0415487E7|
|1740|        青海省| 2.086058E7|
|1271|        天津市|2.2176234E7|
|1296|        上海市|2.3644975E7|
|1760|        北京市|3.0573731E7|
|2347|        重庆市|3.7011754E7|
|3109|        吉林省|3.9025639E7|
|3247|        甘肃省|4.2083765E7|
|4709|      黑龙江省|5.1785284E7|
|3857|        贵州省|6.0030273E7|
|3871|        江西省|6.1545331E7|
|3888|        福建省|6.1804045E7|
|4180|广西壮族自治区|6.3315192E7|
|4278|        陕西省|6.6152502E7|
|4201|        山西省|6.6718433E7|
|4789|        云南省|6.9052626E7|
|4580|        辽宁省|6.9871914E7|
+----+--------------+-----------+
only showing top 20 rows



In [19]:
df_raw = spark.read.format("com.databricks.spark.csv") \
.option("delimiter", "\t") \
.load("/Users/andyzhuang/tmp/link_grid_lenth_zoom15_version2.csv")

df_raw.show(5)
df_raw.createTempView("tmp_link_grid_elem") 

+-------------+------+----+---------------+---+
|          _c0|   _c1| _c2|            _c3|_c4|
+-------------+------+----+---------------+---+
|     13194124|610429|西北|310231212030203|364|
|9100051363998|610528|西北|310231211231222| 15|
|9100042483150|610827|西北|310233120101302| 11|
|9100055577627|610524|西北|310231302301223| 47|
|9100042071344|610326|西北|310231023233022|224|
+-------------+------+----+---------------+---+
only showing top 5 rows



In [36]:
zoom12_le40 = spark.sql("select * from \
                            (select key k12, sum(_c4) len,concat(substr(max(_c1),0,2), '0000') code,max(_c2) region from \
                            (select substr(_c3,0,12) key, * from tmp_link_grid_elem)a \
                            group by key )a \
                            where len<=40*1000 \
                            order by len desc")
zoom12_le40 = zoom12_le40.alias("zoom12_le40")
zoom12_le40.show(5)


+------------+-------+------+------+
|         k12|    len|  code|region|
+------------+-------+------+------+
|310211232321|39999.0|520000|  西南|
|312112001120|39997.0|220000|  东北|
|312130003221|39996.0|150000|  东北|
|310033332321|39994.0|440000|  华南|
|310302231210|39992.0|420000|  华中|
+------------+-------+------+------+
only showing top 5 rows



In [53]:
t_df = spark.sql("select substr(_c3,0,12) k12, substr(_c3,0,13) k13,substr(_c3,0,14) k14,* from tmp_link_grid_elem")
t_df = t_df.alias("t_df")
t_df.show(5)



+------------+-------------+--------------+-------------+------+----+---------------+---+
|         k12|          k13|           k14|          _c0|   _c1| _c2|            _c3|_c4|
+------------+-------------+--------------+-------------+------+----+---------------+---+
|310231212030|3102312120302|31023121203020|     13194124|610429|西北|310231212030203|364|
|310231211231|3102312112312|31023121123122|9100051363998|610528|西北|310231211231222| 15|
|310233120101|3102331201013|31023312010130|9100042483150|610827|西北|310233120101302| 11|
|310231302301|3102313023012|31023130230122|9100055577627|610524|西北|310231302301223| 47|
|310231023233|3102310232330|31023102323302|9100042071344|610326|西北|310231023233022|224|
+------------+-------------+--------------+-------------+------+----+---------------+---+
only showing top 5 rows



In [56]:
# k13


zoom13_le40 = t_df.join(zoom12_le40, zoom12_le40.k12==t_df.k12, 'left').where("zoom12_le40.k12 is null").select("t_df.*") \
.groupby(t_df.k13).agg({'_c4': 'sum', '_c1': 'min', '_c2': 'min'}).toDF('k13', 'len', 'code', 'region') \
.where("len<=40*1000")
zoom13_le40 = zoom13_le40.orderBy(zoom13_le40.len.desc())
zoom13_le40 = zoom13_le40.alias("zoom13_le40")
zoom13_le40.show(5)



+-------------+-------+------+------+
|          k13|    len|  code|region|
+-------------+-------+------+------+
|3102232133211|40000.0|620200|  西北|
|3102311213031|39999.0|611023|  西北|
|3103223301221|39996.0|110117|  华北|
|3103230001220|39995.0|370781|  华北|
|3102102102232|39994.0|530103|  西南|
|3103210101201|39990.0|321283|  华东|
|3121101222302|39989.0|220282|  东北|
|3103212033122|39983.0|371102|  华北|
|3103032333210|39977.0|320582|  华东|
|3121122122211|39977.0|230223|  东北|
|3103032131232|39972.0|330481|  华东|
|3103221113120|39966.0|370321|  华北|
|3102121332130|39965.0|500116|  西南|
|3102332223100|39964.0|150802|  东北|
|3102312212001|39962.0|621002|  西北|
|3103032133230|39960.0|330402|  华东|
|3103202103023|39957.0|410204|  华中|
|3103003110111|39953.0|350603|  华东|
|3103201312032|39952.0|341323|  华中|
|3103221100302|39951.0|370105|  华北|
+-------------+-------+------+------+
only showing top 20 rows



In [58]:
zoom13_le40 = zoom13_le40.alias("zoom13_le40")
# k14
zoom14_le40 = t_df.join(zoom13_le40, zoom13_le40.k13==t_df.k13, 'left').where("zoom13_le40.k13 is null") \
.select("t_df.*") \
.groupby(t_df.k14).agg({'_c4': 'sum', '_c1': 'min', '_c2': 'min'}).toDF('k14', 'len', 'code', 'region') \
.where("len<=40*1000")
zoom14_le40 = zoom14_le40.orderBy(zoom14_le40.len.desc())
zoom14_le40 = zoom14_le40.alias("zoom14_le40")
zoom14_le40.show(5)




+--------------+-------+------+------+
|           k14|    len|  code|region|
+--------------+-------+------+------+
|31030303202001|39995.0|330702|  华东|
|31032022213102|39992.0|140402|  华北|
|31032202002003|39992.0|140107|  华北|
|31021231112123|39987.0|500112|  西南|
|31030200232011|39985.0|430104|  华中|
+--------------+-------+------+------+
only showing top 5 rows



In [61]:
zoom14_le40 = zoom14_le40.alias("zoom14_le40")
# k15
zoom15_le40 = t_df.join(zoom14_le40, zoom14_le40.k14==t_df._c3, 'left').where("zoom14_le40.k14 is null") \
.select("t_df.*") \
.groupby(t_df._c3).agg({'_c4': 'sum', '_c1': 'min', '_c2': 'min'}).toDF('k15', 'len', 'code', 'region') \
.where("len<=40*1000")
zoom15_le40 = zoom15_le40.orderBy(zoom15_le40.len.desc())
zoom15_le40 = zoom15_le40.alias("zoom15_le40")
zoom15_le40.show(5)

+---------------+-------+------+------+
|            k15|    len|  code|region|
+---------------+-------+------+------+
|310300003221210|36658.0|440104|  华南|
|310300010223121|34310.0|440304|  华南|
|310300010223103|33163.0|440304|  华南|
|310303320330020|31185.0|310101|  华东|
|310303032312031|30720.0|330782|  华东|
+---------------+-------+------+------+
only showing top 5 rows



In [62]:
# union
result_df = zoom12_le40.toDF('key','len', 'code', 'region') \
.union(zoom13_le40.toDF('key','len', 'code', 'region')) \
.union(zoom14_le40.toDF('key','len', 'code', 'region')) \
.union(zoom15_le40.toDF('key','len', 'code', 'region')) \



In [63]:
c = result_df.count()
print(c)
result_df.agg({'len': 'sum'}).show()

1376826
+-------------+
|     sum(len)|
+-------------+
|4.847289349E9|
+-------------+

